In [ ]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
sns.set(style="whitegrid",
        rc={
            "text.color": "black",
            "axes.labelcolor": "black",
            "axes.edgecolor": "black",
            "xtick.color": "black",
            "ytick.color": "black",
            "axes.facecolor": "#FFFFFF",
            "figure.facecolor": "#F5F8FA"}
        )

In [ ]:
df=pd.read_pickle("final_df.pkl").reset_index(drop=True)

In [ ]:
df[["tweet","usuario","dispositivo"]].describe()

In [ ]:
round(df["hashtags"].apply(len).mean(),0)

In [ ]:
round(df["users"].apply(len).mean(),0)

In [ ]:
df["dispositivo"].value_counts()[:10]

In [ ]:
hashtags=[]
for i in df["hashtags"]:
    for j in i:
        hashtags.append(j)


In [ ]:
join_hash=(",").join(hashtags).lower()

In [ ]:
reg_amlo=re.compile(r"#amlo,")
clean_hash=reg_amlo.sub("",join_hash).split(",")

In [ ]:
clean_hash.pop(67417)

In [ ]:
hashtag_series=pd.Series(clean_hash)

In [ ]:
hashtag_series.nunique()

In [ ]:
top_hash=hashtag_series.value_counts().head(30)

In [ ]:
plt.figure(figsize=(20,10))
sns.barplot(x=top_hash.values, y=top_hash.index, palette="GnBu_r",lw=1,edgecolor="black")
plt.yticks(size=20,weight="light")
plt.xticks(size=20,weight="light")
plt.xlabel("Número de tweets",size=22,labelpad=20,weight="bold")
plt.title("Hashtags más usados",weight="bold",pad=15,size=25)
plt.tight_layout()

In [ ]:
mentions=[]
for i in df["users"]:
    for j in i:
        mentions.append(j)

In [ ]:
mentions_series=pd.Series(mentions)

In [ ]:
mentions_series.nunique()

In [ ]:
top_mentions=mentions_series.value_counts().head(30)

In [ ]:
plt.figure(figsize=(20,10))
sns.barplot(x=top_mentions.values, y=top_mentions.index, palette="GnBu_r",lw=1,edgecolor="black")
plt.yticks(size=20,weight="light")
plt.xticks(ticks=np.arange(0,6000,500),size=20,weight="light")
plt.xlabel("Número de tweets",size=22,labelpad=20,weight="bold")
plt.title("Usuarios más mencionados",weight="bold",pad=15,size=25)
plt.tight_layout()

In [ ]:
from nltk.corpus import stopwords
stop=stopwords.words('spanish')

In [ ]:
def quitar_acentos(texto):
    juntar=(" ").join(texto)
    juntar=juntar.replace("á","a")
    juntar=juntar.replace("é","e")
    juntar=juntar.replace("í","i")
    juntar=juntar.replace("ó","o")
    juntar=juntar.replace("ú","u")
    lista_final=juntar.split()
    return lista_final
    

In [ ]:
stop=quitar_acentos(stop)

In [ ]:
def texto_a_palabras(texto):
    return [palabra for palabra in texto.split() if palabra not in stop]

In [ ]:
df["tokens"]=df["tweet"].apply(texto_a_palabras)

In [ ]:
conteo={}
for i in df["tokens"]:
    for palabra in i:
        if palabra in conteo:
            conteo[palabra] += 1
        else:
            conteo[palabra] = 1

In [ ]:
conteo_palabras=pd.Series(conteo)

In [ ]:
conteo_palabras.sort_values(ascending=False).head(30)

In [ ]:
lista_tokens=[]
for i in df["tokens"]:
    for palabra in i:
        lista_tokens.append(palabra)

In [ ]:
palabras=(" ").join(lista_tokens)

In [ ]:
wordcloud = WordCloud(height=3000,width=5500,background_color="white",max_words=500).generate(palabras)
plt.figure(figsize=(25,15))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
with open("positive_words_es.txt", "r", encoding="utf-8") as f:
    palabras_positivas = f.read().splitlines()

with open("negative_words_es.txt", "r", encoding="utf-8") as f:
    palabras_negativas = f.read().splitlines()
    
with open("groserías.txt", "r", encoding="utf-8") as f:
    groserias = f.read().splitlines() 

In [ ]:
palabras_positivas=quitar_acentos(palabras_positivas)
palabras_negativas=quitar_acentos(palabras_negativas)

In [ ]:
len(palabras_positivas)

In [ ]:
len(palabras_negativas)

In [ ]:
len(groserias)

In [ ]:
conteo_palabras.count()

In [ ]:
mes_contador=df.groupby(["mes","dia"])["tweet"].count()

In [ ]:
mes_contador.mean()

In [ ]:
hora_contador=df.groupby("hora")["tweet"].count()

In [ ]:
figure=plt.figure(figsize=(20,10))
ax1= figure.add_subplot(1,2,1)
ax2=figure.add_subplot(1,2,2,sharey = ax1)
sns.barplot(x=df["fecha_str"].unique(),color="#1DA1F2",y=mes_contador.values,lw=1,edgecolor="black",orient="v",ax=ax1)
ax1.tick_params(axis="x", labelsize=20,labelrotation=90)
ax1.tick_params(axis="y", labelsize=20)
ax1.set_title("Tweets por día",fontsize=25,fontweight="bold",pad=20)
ax1.set_xlabel("Fecha",fontsize=20,fontweight="bold",labelpad=20)
ax1.set_ylabel("Número de tweets",fontsize=20,fontweight="bold",labelpad=20)
sns.barplot(x=hora_contador.index,color="#1DA1F2",y=hora_contador.values,lw=1,edgecolor="black",orient="v",ax=ax2)
plt.setp(ax2.get_yticklabels(), visible=False)
ax2.tick_params(axis="x", labelsize=20)
ax2.tick_params(axis="y", labelsize=20)
ax2.set_title("Tweets acumulados por hora",fontsize=25,fontweight="bold",pad=20)
ax2.set_xlabel("Tiempo del centro, horario de verano (UTC -5)",fontsize=20,fontweight="bold",labelpad=20)
figure.tight_layout()
plt.subplots_adjust(wspace = 0.05)


In [ ]:
def puntaje(texto):
    score=0
    for i in texto:
        if i in palabras_positivas:
            score+=1
        elif i in palabras_negativas:
            score-=1
    return score

In [ ]:
def contador_groserias(texto):
    score_groserias=0
    for i in texto:
        if i in groserias:
            score_groserias+=1
    return score_groserias

In [ ]:
df["puntaje"]=df["tokens"].apply(puntaje)

In [ ]:
df["conteo_groserias"]=df["tokens"].apply(contador_groserias)

In [ ]:
def multiplicador(punt,gros):
    if gros==1:
        puntaje=punt*2
    elif gros>1:
        puntaje=punt*3
    else:
        puntaje=punt
    return puntaje
    

In [ ]:
df["puntaje_final"]=df.apply(lambda x: multiplicador(x["puntaje"],x["conteo_groserias"]),axis=1)

In [ ]:
df_puntaje=df[df["puntaje_final"]!=0]

In [ ]:
df_puntaje["sentimiento"]=df["puntaje_final"].apply(lambda x: "positivo" if x>0 else "negativo")

In [ ]:
df_puntaje["sentimiento"].value_counts()

In [ ]:
df_sent=df_puntaje.groupby(["mes","dia","sentimiento"]).count()["tweet"]

In [ ]:
df_sent.xs("positivo",level=2).plot(lw=2,ls="--",color="green",figsize=(20,10),label="Positivo")
df_sent.xs("negativo",level=2).plot(lw=2,ls="--",color="red",figsize=(20,10),label="Negativo")
plt.xticks(ticks=np.arange(0, df_puntaje["fecha_str"].nunique()),labels=df_puntaje["fecha_str"].unique(),size=20,rotation=90)
plt.yticks(np.arange(0, 851 , 50),size=20)
plt.title("Sentimiento de los tweets",weight="bold",pad=15,size=22)
plt.xlabel("Fecha",weight="bold",labelpad=20,size=18)
plt.ylabel("Número de tweets",weight="bold",labelpad=20,size=18)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5),fontsize="x-large")
plt.tight_layout()

In [ ]:
df_suma=df_puntaje.groupby(["mes","dia"])["puntaje"].sum()

In [ ]:
def indice(x):
    if x>0:
        valor=x/df_suma.max()
    if x<0:
        valor=-(x/df_suma.min())
    return valor

In [ ]:
df_indice=df_suma.apply(indice)

In [ ]:
df_indice.plot(lw=2,ls="--",color="red",figsize=(20,10))
plt.xticks(ticks=np.arange(0, df_puntaje["fecha_str"].nunique()),labels=df_puntaje["fecha_str"].unique(),size=15,rotation=90)
plt.title("Sentimiento general diario de los tweets",weight="bold",pad=15,size=22)
plt.xlabel("Fecha",weight="bold",labelpad=20,size=18)
plt.yticks(size=15)
plt.ylabel("Índice",weight="bold",labelpad=20,size=18)